<a href="https://colab.research.google.com/github/Hidenori24/LearnDirectory/blob/master/SMBC2025_ver2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 0. ライブラリセットアップ


In [10]:
# ============================================
# 0. ライブラリ & CFG 定義
# ============================================
!pip install -q xgboost optuna japanize-matplotlib folium

import os, warnings, random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import japanize_matplotlib
import folium
from xgboost import XGBRegressor
import optuna
from sklearn.metrics import mean_squared_error, r2_score

warnings.filterwarnings("ignore")

# ---------- CFG ----------
class CFG:
    seed         = 42

    data_path    = '/content/drive/MyDrive/ML/Signate_1634/'
    corr_threshold = 0.13                 # 相関係数の閾値
    comfort_index  = 65                   # 不快指数の快適基準
    optuna_trials  = 50                   # Optuna 試行回数


# set seed
random.seed(CFG.seed)
np.random.seed(CFG.seed)


# 1. Google Drive マウント


In [11]:
# ============================================
# 1. Google Drive マウント
# ============================================
from google.colab import drive
drive.mount('/content/drive', force_remount=True)


Mounted at /content/drive


# 2. データ読み込み

In [12]:
# =========================================================
# 2. データ読み込み
#    - index を DatetimeIndex（UTC）に
# =========================================================
def read_data(path: str) -> pd.DataFrame:
    df = pd.read_csv(path)                 # まずは普通に読み込み
    df['time'] = pd.to_datetime(df['time'], utc=True)   # ①文字列→datetime(UTC)
    df['time'] = df['time'].dt.tz_convert(None)         # ②タイムゾーン情報を外す（naive へ）
    df = df.set_index('time').sort_index()              # ③DatetimeIndex として設定
    return df

train_df = read_data(os.path.join(CFG.data_path, 'train.csv'))
test_df  = read_data(os.path.join(CFG.data_path, 'test.csv'))

# タイムゾーンを UTC→Etc/GMT-1 に変換
train_df.index = pd.to_datetime(train_df.index, utc=True).tz_convert('Etc/GMT-1')
test_df.index = pd.to_datetime(test_df.index,  utc=True).tz_convert('Etc/GMT-1')

print('train', train_df.shape, 'test', test_df.shape)



train (26280, 91) test (8760, 90)


#3 前処理

In [13]:
import holidays
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import TimeSeriesSplit
import lightgbm as lgb

# weather_main, weather_icon, weather_id 列の削除
for kw in ['weather_main','weather_icon','weather_id']:
    cols = [c for c in train_df.columns if kw in c]
    train_df.drop(columns=cols, inplace=True)
    test_df.drop(columns=cols, inplace=True)

# 値がすべて同じ列の削除
constant_cols = [c for c in train_df.columns if train_df[c].nunique() == 1]
train_df.drop(columns=constant_cols, inplace=True)
test_df.drop(columns=constant_cols, inplace=True)

# 大気圧のクリッピング（1090 hPa で上限）
pressure_cols = [c for c in train_df.columns if 'pressure' in c]
train_df[pressure_cols] = train_df[pressure_cols].clip(upper=1090)
test_df [pressure_cols] = test_df [pressure_cols].clip(upper=1090)

# 風速のクリッピング（18 m/s で上限）
wind_cols = [c for c in train_df.columns if 'wind_speed' in c]
train_df[wind_cols] = train_df[wind_cols].clip(upper=18)
test_df [wind_cols] = test_df [wind_cols].clip(upper=18)

# 欠損値を前方埋め、後方埋め
train_df.ffill(inplace=True); train_df.bfill(inplace=True)
test_df .ffill(inplace=True); test_df .bfill(inplace=True)

# 4. 不快指数 → 差分絶対値

In [14]:
#  大気圧の上下クリップ (下限900hPa, 上限1090hPa)
pressure_columns = [col for col in train_df.columns if 'pressure' in col]
train_df[pressure_columns] = train_df[pressure_columns].clip(lower=900, upper=1090)
test_df [pressure_columns] = test_df [pressure_columns].clip(lower=900, upper=1090)


def discomfort_index(temp_k, hum):
    # ケルビン→摂氏変換
    temp_c = temp_k - 273.15
    # temp_c * 0.99 - 14.3 を使った不快指数
    return temp_c * 0.81 + (hum / 100) * (temp_c * 0.99 - 14.3) + 46.3

for city in ['valencia','madrid','bilbao','barcelona','seville']:
    # “生”不快指数
    train_df[f'di_{city}']      = discomfort_index(train_df[f'{city}_temp'], train_df[f'{city}_humidity'])
    test_df [f'di_{city}']      = discomfort_index(test_df [f'{city}_temp'], test_df [f'{city}_humidity'])
    # 快適基準CFG.comfort_indexとの差分絶対値
    train_df[f'di_{city}_diff_abs'] = (train_df[f'di_{city}'] - CFG.comfort_index).abs()
    test_df [f'di_{city}_diff_abs'] = (test_df [f'di_{city}'] - CFG.comfort_index).abs()

# 元の temp, humidity, “生”不快指数列は不要なので削除
drop_cols = [
    c for c in train_df.columns
    if ('_temp' in c) or ('_humidity' in c) or (c.startswith('di_') and not c.endswith('_diff_abs'))
]
train_df.drop(columns=drop_cols, inplace=True)
test_df .drop(columns=drop_cols, inplace=True)

# ============================================
# 5. generation_sum を追加
# ============================================
gen_cols = [c for c in train_df.columns if c.startswith('generation')]
train_df['generation_sum'] = train_df[gen_cols].sum(axis=1)
test_df ['generation_sum'] = test_df [gen_cols].sum(axis=1)

# ============================================
# 6. 相関係数で特徴量選択（数値列のみで計算）
# ============================================
# 数値列だけを抽出
numeric_cols = train_df.select_dtypes(include=[np.number]).columns

# 数値列同士の相関行列を計算し、price_actual との相関を取得
corr = train_df[numeric_cols].corr()['price_actual'].abs()

# 閾値以上かつ price_actual 自身は除外
use_cols = corr[corr > CFG.corr_threshold].index.drop('price_actual').tolist()
print("選択された特徴量:", use_cols)

# ============================================
# 7. 日付情報の追加 & One-Hot
# ============================================
for df in [train_df, test_df]:
    df.index = pd.to_datetime(df.index)  # 念のため
    df['month']   = df.index.month       # ←ここを追加！
    df['hour']    = df.index.hour
    df['weekday'] = df.index.dayofweek
    df['day_cat'] = df.index.day.map(lambda d: 0 if d <= 10 else 1 if d <= 20 else 2)
# month は one-hot に
train_df = pd.get_dummies(train_df, columns=['month'], prefix='month', drop_first=True)
test_df  = pd.get_dummies(test_df,  columns=['month'], prefix='month', drop_first=True)

選択された特徴量: ['generation_biomass', 'generation_fossil_brown_coal/lignite', 'generation_fossil_gas', 'generation_fossil_hard_coal', 'generation_fossil_oil', 'generation_hydro_pumped_storage_consumption', 'generation_hydro_run_of_river_and_poundage', 'total_load_actual', 'valencia_wind_speed', 'madrid_wind_speed', 'bilbao_pressure', 'bilbao_wind_speed', 'bilbao_clouds_all', 'barcelona_pressure', 'barcelona_wind_speed', 'seville_pressure', 'seville_wind_deg', 'di_valencia_diff_abs', 'generation_sum']


# 8.学習／検証データの準備

In [15]:
X = train_df[use_cols + ['hour','weekday','day_cat'] + [c for c in train_df.columns if c.startswith('month_')]]
y = train_df['price_actual']
X_test = test_df[X.columns]  # テストデータの列順を合わせる

X_train = X[train_df.index.year < 2017]
y_train = y[train_df.index.year < 2017]
X_val   = X[train_df.index.year == 2017]
y_val   = y[train_df.index.year == 2017]

print("train:", X_train.shape, "val:", X_val.shape, "test:", X_test.shape)

train: (17520, 33) val: (8760, 33) test: (8760, 33)


# 9. 年度別分割とアンサンブル学習

In [16]:
# 各年度を定義
years = sorted(train_df.index.year.unique()) # 例: [2015, 2016, 2017]

# 各分割での予測結果を格納するリスト
test_predictions = []

# 3通りの分割でモデルを学習
for held_out_year in years:
    print(f"\n--- 学習: 検証年度 {held_out_year} ---")

    # 学習データと検証データの分割
    train_years = [y for y in years if y != held_out_year]
    X_train_fold = X[X.index.year.isin(train_years)]
    y_train_fold = y[y.index.year.isin(train_years)]
    X_val_fold   = X[X.index.year == held_out_year]
    y_val_fold   = y[y.index.year == held_out_year]

    print(f"学習データ ({train_years}):", X_train_fold.shape, "検証データ ({held_out_year}):", X_val_fold.shape)

    # Optunaによるパラメータチューニング（各分割ごとに実行）
    # 注意: 各分割でOptunaを実行すると計算時間が大幅に増加します。
    # 計算時間を抑えたい場合は、一度全体でOptunaを実行したベストパラメータを使用するか、
    # Optunaの試行回数を減らすなどを検討してください。
    def fold_objective(trial):
        params = {
            'objective':        'reg:quantileerror',
            'eval_metric':      'rmse',
            'learning_rate':    trial.suggest_float('learning_rate', 0.001, 0.2),
            'n_estimators':     trial.suggest_int('n_estimators', 100, 1000),
            'max_depth':        trial.suggest_int('max_depth', 5, 10),
            'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
            'gamma':            trial.suggest_float('gamma', 0.001, 0.1),
            'subsample':        trial.suggest_float('subsample', 0.5, 1.0),
            'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
            'lambda':           trial.suggest_float('lambda', 0.001, 1.0),
            'quantile_alpha':   trial.suggest_float('quantile_alpha', 0.7, 0.9),
            'verbosity':        0,
            'random_state':     CFG.seed
        }
        model = XGBRegressor(**params)
        model.fit(X_train_fold, y_train_fold)
        preds = model.predict(X_val_fold)
        rmse = np.sqrt(mean_squared_error(y_val_fold, preds))
        return rmse

    # 各分割でOptunaを実行する場合
    # study_fold = optuna.create_study(direction='minimize',
    #                                 sampler=optuna.samplers.TPESampler(seed=CFG.seed))
    # study_fold.optimize(fold_objective, n_trials=CFG.optuna_trials)
    # best_params_fold = study_fold.best_params
    # print(f"検証年度 {held_out_year} のベストパラメータ:", best_params_fold)

    # 計算時間削減のため、ここでは全体のOptunaで得られたbest_paramsを使用すると仮定
    # もし各分割でOptunaを実行したい場合は、上記のコメントアウトを解除し、
    # best_params の代わりに best_params_fold を使用してください。
    best_params_fold = best_params # 例: 全体で得られたbest_paramsを使用

    # ベストパラメータでモデルを学習（学習データ全体を使用）
    model = XGBRegressor(
        objective='reg:quantileerror',
        eval_metric='rmse',
        verbosity=0,
        random_state=CFG.seed,
        **best_params_fold
    )
    # 注意: アンサンブルのため、ここでは学習データ全体(X_train_fold, y_train_fold)ではなく、
    # 該当の学習期間で学習します。
    model.fit(X_train_fold, y_train_fold)

    # テストデータの予測
    fold_test_pred = model.predict(X_test)
    test_predictions.append(fold_test_pred)
    # この分割でのテスト予測結果を submission ファイルとして出力
    sub_fold = pd.read_csv(CFG.data_path + 'sample_submit.csv', header=None)
    sub_fold[1] = fold_test_pred
    output_filename = f'submission_held_out_{held_out_year}.csv'
    sub_fold.to_csv(output_filename, header=False, index=False)
    print(f"  >>> {output_filename} を出力しました")

# アンサンブル（ここでは平均をとる）
y_test_pred = np.mean(test_predictions, axis=0)


--- 学習: 検証年度 2015 ---
学習データ ([2016, 2017]): (17544, 33) 検証データ ({held_out_year}): (8736, 33)
  >>> submission_held_out_2015.csv を出力しました

--- 学習: 検証年度 2016 ---
学習データ ([2015, 2017]): (17496, 33) 検証データ ({held_out_year}): (8784, 33)
  >>> submission_held_out_2016.csv を出力しました

--- 学習: 検証年度 2017 ---
学習データ ([2015, 2016]): (17520, 33) 検証データ ({held_out_year}): (8760, 33)
  >>> submission_held_out_2017.csv を出力しました


# 10. ベストパラメータで最終モデル学習 → テスト予測


In [17]:
# 欠番
# model = XGBRegressor(
#     objective='reg:quantileerror',
#     eval_metric='rmse',
#     verbosity=0,
#     random_state=CFG.seed,
#     **best_params
# )
# model.fit(X, y)
# y_test_pred = model.predict(X_test)

# 11. Submission ファイル出力

In [18]:
sub = pd.read_csv(CFG.data_path + 'sample_submit.csv', header=None)
sub[1] = y_test_pred
sub.to_csv('submission.csv', header=False, index=False)
print("\n>>> submission.csv を出力しました")


>>> submission.csv を出力しました
